In [5]:
import cv2
import mediapipe as mp
from cvzone.HandTrackingModule import HandDetector
import socket
import json

cap = cv2.VideoCapture(0) # uncomment if you have webcam
# cap = cv2.VideoCapture("http://10.0.0.115:4747/video") # ONLY OPENS UP WHEN ONE INSTANCE OF THIS IS OPEN! DO NOT OPEN IN BROWSER!
IMG_HEIGHT = 720
IMG_WIDTH = 1280 
cap.set(3, IMG_WIDTH)
cap.set(4, IMG_HEIGHT)
if not cap.isOpened():
    raise IOError("Cannot open webcam")
    
# DRAW BOTH HANDS ON SCREEN
detector = HandDetector(maxHands=2, detectionCon=0.8)

# Comms
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
serverAddressPort = ("127.0.0.1", 5052)

while(True):
    success, img = cap.read()
    # flipping image to make everything easier to control (like a mirror)

    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)
    
    """
    Structure: {
        "landmarks": List,
        "bbox": Tuple,
        "center": Tuple,
        "type": String
    }
    """
    data = {
        "left": {
            "landmarks": [],
            "bbox": (),
            "center": (),
            "type": ""
        },
        "right": {
            "landmarks": [],
            "bbox": (),
            "center": (),
            "type": ""
        }
    }
    if hands:
        for hand in hands:
            # get left or right from hand so we can access the correct hand in the data object with a single if statement
            lr = hand["type"].lower()

            if (data[lr]["type"] == ""):
                # get list of landmarks, bounding box, center, then add them to JSON object
                lmList = hand["lmList"]
                bbox = hand["bbox"]
                center = hand["center"]

                data[lr]["bbox"] = bbox
                data[lr]["center"] = center
                data[lr]["type"] = lr

                for lm in lmList:
                    data[lr]["landmarks"].extend([lm[0], IMG_HEIGHT - lm[1], lm[2]])

        # convert to JSON
        json_data = json.dumps(data)
        sock.sendto(json_data.encode(), serverAddressPort)
    # if hands:
    #     hand = hands[0]
    #     # get list of landmarks
    #     lmList = hand['lmList']

    #     for lm in lmList:
    #         data.extend([lm[0], IMG_HEIGHT - lm[1], lm[2]])
    #     sock.sendto(str.encode(str(data)), serverAddressPort)
    
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

{"left": {"landmarks": [669, 186, 0, 649, 229, 25, 643, 273, 25, 657, 303, 17, 676, 321, 7, 625, 296, -3, 659, 363, -15, 691, 379, -18, 716, 376, -16, 641, 296, -24, 667, 376, -37, 705, 395, -37, 731, 389, -32, 663, 289, -43, 685, 365, -51, 723, 381, -45, 747, 378, -38, 691, 276, -59, 703, 338, -63, 725, 355, -56, 742, 358, -48], "bbox": [625, 325, 122, 209], "center": [686, 429], "type": "left"}, "right": {"landmarks": [], "bbox": [], "center": [], "type": ""}}
{"left": {"landmarks": [657, 179, 0, 648, 228, 30, 645, 277, 36, 661, 306, 33, 679, 325, 29, 622, 305, 13, 655, 368, 4, 688, 383, 0, 713, 382, 0, 635, 306, -9, 661, 382, -18, 699, 399, -19, 725, 394, -15, 656, 299, -30, 679, 371, -36, 717, 387, -31, 740, 384, -25, 684, 286, -49, 698, 347, -50, 721, 363, -42, 738, 366, -34], "bbox": [622, 321, 118, 220], "center": [681, 431], "type": "left"}, "right": {"landmarks": [], "bbox": [], "center": [], "type": ""}}
{"left": {"landmarks": [649, 187, 0, 637, 238, 25, 636, 283, 26, 652, 31

In [7]:
print(hand['lmList'])
print(hand)

[[479, 341, 0], [551, 317, -23], [612, 267, -32], [630, 206, -40], [622, 157, -49], [593, 185, -18], [622, 118, -41], [640, 79, -60], [659, 51, -72], [554, 165, -22], [581, 82, -42], [602, 34, -59], [622, 0, -71], [514, 161, -30], [540, 82, -51], [558, 35, -66], [578, 1, -75], [470, 172, -39], [476, 109, -61], [486, 68, -70], [499, 33, -76]]
{'lmList': [[479, 341, 0], [551, 317, -23], [612, 267, -32], [630, 206, -40], [622, 157, -49], [593, 185, -18], [622, 118, -41], [640, 79, -60], [659, 51, -72], [554, 165, -22], [581, 82, -42], [602, 34, -59], [622, 0, -71], [514, 161, -30], [540, 82, -51], [558, 35, -66], [578, 1, -75], [470, 172, -39], [476, 109, -61], [486, 68, -70], [499, 33, -76]], 'bbox': (470, 0, 189, 341), 'center': (564, 170), 'type': 'Left'}
